In [1]:
import sys
sys.path.append('/Users/samrelins/Documents/LIDA/ace_project/')
from src.train_test import *

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/samrelins/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samrelins/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data_dir = "/Users/samrelins/Documents/LIDA/ace_project/data/"

orig_filename = "ace_data_orig.csv"
ace_dat_orig = pd.read_csv(data_dir + orig_filename)

extra_filename = "ace_data_extra.csv"
ace_dat_extra = pd.read_csv(data_dir + extra_filename)

In [3]:
from sklearn.linear_model import LogisticRegression

def return_log_reg_clf_results(df):
    X_train, y_train, X_test, y_test = return_train_test(df)

    best_scores, best_params = param_search_classifier(
        X_train=X_train, y_train=y_train,
        clf=LogisticRegression(random_state=0,
                               max_iter=1000,
                               solver="saga",
                               class_weight="balanced",
                               n_jobs=-1),
        param_grid={"penalty": ["none", "l1", "l2"],
                    'C': np.logspace(-4, 4, 40)},
        scaled=True,
    )

    cv_scores = pd.DataFrame(best_scores, index=[""]).T

    X_train_prepped, X_test_prepped = encode_and_scale(
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        cat_encoder="one_hot",
        scaled=True
    )

    log_reg_clf = LogisticRegression(
        random_state=0,
        max_iter=1000,
        solver="saga",
        class_weight="balanced",
        n_jobs=-1,
        **best_params
    )

    log_reg_clf.fit(X_train_prepped, y_train)

    test_scores = pd.DataFrame(
        score_classifier(log_reg_clf, X_test_prepped, y_test),
        index=[""]).T

    param_names = X_train_prepped.columns
    coefs = log_reg_clf.coef_[0]
    nonzero_coefs = coefs > 0

    model_coeffs = pd.DataFrame({
        "Feature": param_names[nonzero_coefs],
        "Coefficient Value": coefs[nonzero_coefs]
    }).sort_values("Coefficient Value", ascending=False)
    return cv_scores, test_scores, model_coeffs


In [4]:
return_log_reg_clf_results(ace_dat_orig)

Testing LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1,
                   random_state=0, solver='saga') classifier with one_hot encoded features.


100%|██████████| 120/120 [00:43<00:00,  2.77it/s]


(                    
 f1          0.292178
 roc_auc     0.561783
 accuracy    0.640135
 recall      0.444444
 precision   0.218089
 true_pos    8.000000
 true_neg   61.333333
 false_pos  29.000000
 false_neg  10.000000,
                     
 f1          0.348837
 roc_auc     0.613599
 accuracy    0.652174
 recall      0.555556
 precision   0.254237
 true_pos   15.000000
 true_neg   90.000000
 false_pos  44.000000
 false_neg  12.000000,
                       Feature  Coefficient Value
 2                address_bd02           1.312936
 14         gut_feeling_unwell           1.166088
 28                  resp_rate           1.011765
 25   apls_heart_rate_cat_high           0.972487
 20        age_range_secondary           0.912898
 8                address_bd19           0.884143
 3                address_bd03           0.770143
 1            referral_from_gp           0.724753
 24    apls_resp_rate_cat_high           0.705556
 12      referral_time_evening           0.659623
 29     

In [6]:
referral_from_dict =  {"GP": "GP",
                       "A&E": "ED",
                       "GP ANP": "GP",
                       "CCDA": "CCDA",
                       "ED Paed ANP": "ED",
                       "CCDA ANP": "ANP",
                       "GP - Advanced Pharmacist": "GP",
                       "None": "None"}

ace_dat_extra["referral_from"] = ace_dat_extra.referral_from.apply(
    lambda x: referral_from_dict[x]
)

text_features = ["medical_history", "examination_summary", "recommendation"]
return_log_reg_clf_results(ace_dat_extra.drop(text_features, axis=1))

Testing LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1,
                   random_state=0, solver='saga') classifier with one_hot encoded features.


100%|██████████| 120/120 [00:47<00:00,  2.54it/s]


(                    
 f1          0.287737
 roc_auc     0.559524
 accuracy    0.599388
 recall      0.500000
 precision   0.202963
 true_pos    9.000000
 true_neg   56.333333
 false_pos  34.666667
 false_neg   9.000000,
                     
 f1          0.243902
 roc_auc     0.518519
 accuracy    0.617284
 recall      0.370370
 precision   0.181818
 true_pos   10.000000
 true_neg   90.000000
 false_pos  45.000000
 false_neg  17.000000,
                          Feature  Coefficient Value
 0               referral_from_gp           0.064592
 1     referral_profession_doctor           0.062497
 25       apls_resp_rate_cat_high           0.054424
 13          referral_date_winter           0.048370
 15     illness_severity_moderate           0.044401
 18                food_allergy_y           0.036662
 4                   address_bd03           0.036616
 26      apls_heart_rate_cat_high           0.035777
 16       gut_feeling_low concern           0.034121
 22        ace_resp_rate_cat

In [7]:
return_log_reg_clf_results(ace_dat_orig)

Testing LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1,
                   random_state=0, solver='saga') classifier with one_hot encoded features.


100%|██████████| 120/120 [00:43<00:00,  2.74it/s]


(                    
 f1          0.292178
 roc_auc     0.561783
 accuracy    0.640135
 recall      0.444444
 precision   0.218089
 true_pos    8.000000
 true_neg   61.333333
 false_pos  29.000000
 false_neg  10.000000,
                     
 f1          0.348837
 roc_auc     0.613599
 accuracy    0.652174
 recall      0.555556
 precision   0.254237
 true_pos   15.000000
 true_neg   90.000000
 false_pos  44.000000
 false_neg  12.000000,
                       Feature  Coefficient Value
 2                address_bd02           1.312936
 14         gut_feeling_unwell           1.166088
 28                  resp_rate           1.011765
 25   apls_heart_rate_cat_high           0.972487
 20        age_range_secondary           0.912898
 8                address_bd19           0.884143
 3                address_bd03           0.770143
 1            referral_from_gp           0.724753
 24    apls_resp_rate_cat_high           0.705556
 12      referral_time_evening           0.659623
 29     

In [8]:
return_log_reg_clf_results(ace_dat_extra)

Testing LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1,
                   random_state=0, solver='saga') classifier with one_hot encoded features.


100%|██████████| 120/120 [00:54<00:00,  2.20it/s]


(                    
 f1          0.360848
 roc_auc     0.625051
 accuracy    0.683939
 recall      0.537037
 precision   0.272700
 true_pos    9.666667
 true_neg   64.666667
 false_pos  26.000000
 false_neg   8.333333,
                      
 f1           0.277778
 roc_auc      0.555556
 accuracy     0.679012
 recall       0.370370
 precision    0.222222
 true_pos    10.000000
 true_neg   100.000000
 false_pos   35.000000
 false_neg   17.000000,
                        Feature  Coefficient Value
 30        summary_salbutamol_y           0.102189
 29               hist_asthma_y           0.069196
 0             referral_from_gp           0.062577
 28          hist_know_asthma_y           0.061330
 1   referral_profession_doctor           0.061136
 31     recommend_prednisolon_y           0.055370
 24     apls_resp_rate_cat_high           0.049177
 12        referral_date_winter           0.048720
 14   illness_severity_moderate           0.044314
 26               hist_montel_y       

In [6]:
drop_features = ["address", "safeguarding"]

return_log_reg_clf_results(ace_dat_orig.drop(crap_features, axis=1))

Testing LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1,
                   random_state=0, solver='saga') classifier with one_hot encoded features.


100%|██████████| 120/120 [00:34<00:00,  3.51it/s]


(                    
 f1          0.321696
 roc_auc     0.587831
 accuracy    0.596982
 recall      0.574074
 precision   0.224927
 true_pos   10.333333
 true_neg   54.333333
 false_pos  36.000000
 false_neg   7.666667,
                     
 f1          0.278481
 roc_auc     0.550719
 accuracy    0.645963
 recall      0.407407
 precision   0.211538
 true_pos   11.000000
 true_neg   93.000000
 false_pos  41.000000
 false_neg  16.000000,
                       Feature  Coefficient Value
 0            referral_from_gp           0.108796
 17    apls_resp_rate_cat_high           0.073532
 5   illness_severity_moderate           0.071592
 3        referral_date_winter           0.071014
 8              food_allergy_y           0.053468
 18   apls_heart_rate_cat_high           0.049079
 4       referral_time_evening           0.042193
 13        age_range_secondary           0.041912
 16  ace_heart_rate_cat_normal           0.041049
 6     gut_feeling_low concern           0.037606
 14     

In [7]:
return_log_reg_clf_results(ace_dat_extra.drop(crap_features, axis=1))


Testing LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1,
                   random_state=0, solver='saga') classifier with one_hot encoded features.


100%|██████████| 120/120 [00:44<00:00,  2.71it/s]


(                    
 f1          0.353092
 roc_auc     0.625661
 accuracy    0.685015
 recall      0.537037
 precision   0.266642
 true_pos    9.666667
 true_neg   65.000000
 false_pos  26.000000
 false_neg   8.333333,
                      
 f1           0.277778
 roc_auc      0.555556
 accuracy     0.679012
 recall       0.370370
 precision    0.222222
 true_pos    10.000000
 true_neg   100.000000
 false_pos   35.000000
 false_neg   17.000000,
                                    Feature  Coefficient Value
 28                    summary_salbutamol_y           0.221507
 27                           hist_asthma_y           0.137300
 26                      hist_know_asthma_y           0.123395
 29                 recommend_prednisolon_y           0.111481
 4               referral_profession_doctor           0.108163
 8                     referral_date_winter           0.105501
 22                 apls_resp_rate_cat_high           0.104213
 1                         referral_from_gp 